# **Static Stability of Canoes in Water**
## **Buoyancy force and submergence depth**
---
## Objective
In this notebook, we want to develop the mathematics needed to calculate the bouyancy force for non trivial shapes such as canoes. We will use these calculations to provide interactive visualizations to better an understanding of the material and math behind canoes.

### **Simple Case**
---

In order to introduce the concept of buoyancy and how it can be used to calculate submergence depth, we consider the simple model of a cuboid block partly submerged in hydrostatic (still) water:

<p><center> 
<img src="figures/B_fig01.png" width="300">
</center></p> 

Let $V_c$ be the cuboids volume and $V_w$ be its submerged volume. To clear some confusing, $V_w$ is the volume of water displaced by our cuboid. If we let $D_c$ and $D_w$ be the densities (mass per unit volume) of the cube and water, we can determine their respective masses. The mass $M_c$ of the cube remains constant while $M_w$ can very depending on the amount of displaced water. We calculate it as follows: 

$$
M_c = V_c \cdot D_c \quad\quad\quad M_w = V_w \cdot D_w
$$

Applying Archimedes' principle to our model, the bouyancy force is equivalent to the water's displaced mass multiplied by gravity. We should also note the force of gravity acting on our cube is its mass multiplied by gravity and it will "push" in the opposite direction to buoyancy. We calculate the forces $F_g$ and $F_b$ acting on our cube.

$$
F_g = - M_c \cdot g \quad\quad\quad F_b = M_w \cdot g
$$

The total sum of forces will be our measure of if the cube will sink or rise. Positive will be treated as rising up, negative will be treated as sinking deeper with respect to the signed orientation of the forces. 


In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() { if (code_show){ $('div.input').hide(); } else { $('div.input').show(); } code_show = !code_show } 
$( document ).ready(code_toggle);
</script>

Raw code hidden. To show code, click <a href="javascript:code_toggle()">here</a>. <b>To begin the notebook, click Kernel then click Restart & Run All. </b> ''')

In [1]:
import scripts.Buoyancy_Interface as bi 
bi.CubeGraph()

TwoByTwoLayout(children=(FloatSlider(value=1.0, continuous_update=False, description='length (m): ', layout=La…

Output()

<font size = "2"> \*Arrow in graph is only for visualizing the total force\*</font>$\\\\$

You may notice there exists a point where the total force equals zero. This is know as the buoyancy equilibrium or neutral buoyancy zone. It is where the object will neither sink or rise, as the force of gravity and buoyancy are equal in magnitude and cancel eachother out. For our simple cube model we can calculate the depth where this happens.

To set the ideal, we want $F_b + F_c = 0$, rearranging and expanding gives us:

$$
\begin{align}
g\cdot V_c\cdot D_c & = g\cdot V_w \cdot D_w \\
(\text{length} \cdot \text{width} \cdot \text{height}) \cdot D_c & =  (\text{length} \cdot \text{width} \cdot \text{water depth}) \cdot D_w \\
\text{height} \cdot \frac{ D_c}{D_w} & = \text{water depth}
\end{align}
$$

We see that the equilibrium point is a ratio between the density of the cube and water, multiplied by the total height of the cube. 

However this calculation only cancels out nicely because of the simple linear relation between height and volume. Real world objects, or anything curvy presents a challenge to the volume definition where the relations can be non linear. This volume method is only good for simple cases where volume can be easily defined or approximated efficiently.


### **General case**
---

The volume and weight method is an intuiative approach to finding buoyancy, however calculating the submerged volume of an object isn't always easy. In order to rectify this problem we can use an important theorem in calculus called Gauss's Theorem or Divergence Theorem. It changes our problem of finding volume to one of summing over certain values along the surface of our object. Though it is more complex in nature, the calculation is more friendly to approximations. 

**Divergence Theorem:** 
<em>The volume integral of the divergence of a vector field, taken throughout a bounded domain $D$, equals the surface integral of the normal componenet of the vector field taken over the surface $S$ bounding $D$</em>
$$
\int\int\int_D \triangledown \cdot \mathbf{F} \text{ d}D = \int\int_S \mathbf{F}\cdot n \text{ d}S
$$
Lets disect this heavy jargon and put it into terms we can understand. 

#### The Left Side
The left side's triple integral of $\triangledown \cdot \mathbf{F}$ is equivalent to summing up every unit volume multiplied by the change in $F$. For our scenario of hydrostatic water, this $F$ is pressure which increases linearly as we go deeper. Using calculus we know that linear functions have a constant rate of change. This allows us to change the $\triangledown \cdot \mathbf{F} = C$ since the triangle operation is the divergence of $\mathbf{F}$, a type of derivative.

$$
\begin{align}
\int\int\int_D \triangledown \cdot \mathbf{F} \text{ d}D &= C\cdot \int\int\int_D \text{ d}D \\
&= C\cdot (Volume)\\
\end{align}
$$

The change in pressure $C$ is equal to **density** times **gravity**. This is the simple case of using volume to determine the bouyancy force.<br /><br />

#### The Right Side
$$
\int\int_S \mathbf{F}\cdot n \text{ d}S
$$
This right side takes every normal vector $n$ along the surface and multiplies it by the $F$ at that point. After which it sums it all up.
As stated before $F$ is our pressure, which increases linearly as we go deeper. This increase can be felt by anyone who has swam too deep in a pool, or felt lack of pressure when on a flight where you've had your ears "pop". 

<p><center>
    <img src="figures/B_fig02.png" width="600">
</center></p>
<font size = "2"><center>
    The left cube is the volume method, right cube is the normals method
</center></font>

The right cube depicts the vectors (arrows) that will be summed up which have been scaled linearly, based on their depth.



#### General Case Approximation Method
This method allows us to easily approximate the equilibrium of any model by partitioning their surface into small squares. Nearly all objects can be made up of these squares/ quads. This allows us to gather the neccesary data for our approximation. 

By storing the positions, normals, and areas in a table we can use them to calculate the $\mathbf{F}\cdot n$ part for each square. It's important we store the position of each normal as $\mathbf{F}$ is technically a function taking a position as input. The area is also important to store because it determines how *strong* the force acting on that square is. 

<p><center>
    <img src="figures/B_fig03.png" width="300">
</center></p>

| <center><font size="3.5">position</font>   | <center><font size="3.5">normal</font> | <center><font size="3.5">area</font> |
| --- | --- | --- |
| <center><font size="3.5">[0.1, -0.2, -3]</font></center> | <center><font size="3.5">[0, 0.5, 0.866]</font></center> | <center><font size="3.5">0.05</font></center> | 
| <center><font size="3.5">[0.2, -0.3, -3.6]</font></center> | <center><font size="3.5">[0, 0.6, 0.8]</font></center> | <center><font size="3.5">0.087</font></center> | 
| <center><font size="3.5">$\vdots$</font></center> | <center><font size="3.5">$\vdots$</font></center> | <center><font size="3.5">$\vdots$</font></center> |

<font size = "2"><center>
This is a short example table of elements
</center></font>

We can now determine the total force acting on our object, since our $\mathbf{F}$ only depends on depth we only need to use $position.z$ to calculate it. $\mathbf{F} = position.z \cdot D_w \cdot g$. Dot producting this with the $normal$ gives us the $Pressure$, then to get the force we use the $Force$ $/ Area = Pressure$ identity.
    
$$
\sum (position.z \cdot D_w \cdot g) \cdot (normal) \cdot (area) = F_b
$$
    
This is a fairly inexpensive operation. We can calculate $F_b$ for multiple depths allowing us to search for the equilibrium level using brute force methods. For our algorithm we used binary search, by taking the top and bottom of our canoe (plus some) as bounds to search through. By starting at the center we can determine if the total summed force with gravity and bouyancy is positive or negative. Then cut our search interval in half appropriately and repeat starting at the new center. This simple method can approach the equilibrium fairly quickly. 

There are more complex search methods that approach faster, but for simplicity we have used a fairly approachable method.

In [2]:
import scripts.BezierSurf_Interface as bsi
import scripts.Buoyancy_Interface as bi 
widgetLength, widgetWidth, widgetHeight, widgetNames = bsi.GetWidgets() 
bi.CanoeBuoyancy(widgetLength, widgetWidth, widgetHeight, widgetNames)
#Temporary bloat, will find a nice way in the future to simplify this

interactive(children=(FloatSlider(value=7.0, continuous_update=False, description='length (m)', max=10.0, min=…

# Citation
```{bibliography}

Davis, H. F., &amp; Snider, A. D. (2000). 4.9 Introduction to the Divergence Theorem and Stokes' Theorem. In Introduction to vector analysis (Seventh, pp. 257–257). essay, Hawkes Publishing. 

Lima, F. M. (2011). Using surface integrals for checking Archimedes' law of buoyancy. European Journal of Physics, 33(1), 101–113. https://doi.org/10.1088/0143-0807/33/1/009 
````